In [4]:
import os
import re
import hashlib
from pydub import AudioSegment
import yt_dlp

def sanitize_filename(filename):
    """Remove invalid characters from filenames"""
    return re.sub(r'[\\/*?:"<>|]', "", filename)

def download_song(url, start_time=None, end_time=None):
    """Download audio with optional timestamps"""
    os.makedirs("downloads", exist_ok=True)
    unique_id = hashlib.md5(url.encode()).hexdigest()[:8]
    ydl_opts = {
        'format': 'bestaudio/best',
        'outtmpl': f'downloads/{unique_id}_%(title)s.%(ext)s',
        'postprocessors': [{
            'key': 'FFmpegExtractAudio',
            'preferredcodec': 'mp3',
            'preferredquality': '320',
        }],
        'quiet': True,
        'nocheckcertificate': True,
    }
    try:
        with yt_dlp.YoutubeDL({'quiet': True}) as ydl:
            info = ydl.extract_info(url, download=False)
            duration = info.get('duration') or 0
        if start_time and start_time > duration:
            print(f"⚠️ Start time {start_time}s exceeds video duration {duration}s. Using start=0.")
            start_time = 0
        if end_time and end_time > duration:
            print(f"⚠️ End time {end_time}s exceeds video duration {duration}s. Using end=video duration.")
            end_time = duration
    except Exception as e:
        print(f"⚠️ Pre-check failed: {str(e)}")
        return None
    if start_time or end_time:
        ydl_opts['postprocessor_args'] = []
        if start_time:
            ydl_opts['postprocessor_args'].extend(['-ss', str(start_time)])
        if end_time:
            ydl_opts['postprocessor_args'].extend(['-to', str(end_time)])
    try:
        with yt_dlp.YoutubeDL(ydl_opts) as ydl:
            info = ydl.extract_info(url, download=True)
            original_filename = ydl.prepare_filename(info)
            base_name = os.path.basename(original_filename)
            mp3_files = [f for f in os.listdir("downloads") if f.startswith(base_name.split('.')[0])]
            if not mp3_files:
                print("❌ Downloaded file not found.")
                return None
            mp3_file = os.path.join("downloads", mp3_files[0])
            clean_name = sanitize_filename(os.path.basename(mp3_file))
            final_path = os.path.join("downloads", clean_name)
            os.rename(mp3_file, final_path)
            return final_path
    except Exception as e:
        print(f"❌ Download failed: {str(e)}")
        return None

def create_transition(segment1, segment2, transition_duration=5000):
    """Create smooth transition between segments with integer duration"""
    transition_duration = int(transition_duration)
    segment1 = segment1.fade_out(transition_duration)
    segment2 = segment2.fade_in(transition_duration)
    return segment1.append(segment2, crossfade=transition_duration)

def create_mashup(song_data_list, output_filename):
    """Create mashup with clean transitions only"""
    os.makedirs("output", exist_ok=True)
    processed_segments = []
    for i, data in enumerate(song_data_list):
        print(f"\nProcessing Song {i+1}")
        audio_path = download_song(data['url'], data.get('start'), data.get('end'))
        if not audio_path:
            print(f"Skipping song {i+1} due to download error")
            continue
        audio = AudioSegment.from_file(audio_path)
        print(f"Loaded '{audio_path}' - Duration: {len(audio)/1000:.2f}s")
        processed_segments.append({'audio': audio, 'title': f"Song {i+1}"})
    if len(processed_segments) < 2:
        raise Exception("Need at least 2 valid songs to create mashup.")
    final_mashup = processed_segments[0]['audio']
    for i in range(1, len(processed_segments)):
        print(f"\nCreating transition between segments {i} and {i+1}")
        transition_duration = min(8000, len(final_mashup) // 4)
        final_mashup = create_transition(
            final_mashup,
            processed_segments[i]['audio'],
            transition_duration
        )
    final_mashup = final_mashup.normalize()
    mashup_path = os.path.join("output", output_filename)
    final_mashup.export(mashup_path, format="mp3", bitrate="320k")
    print(f"\n🎉 Clean mashup created successfully at: {os.path.abspath(mashup_path)}")
    return mashup_path

def parse_timestamp(timestamp):
    """Convert timestamp string (MM:SS or seconds) to seconds"""
    if not timestamp:
        return None
    if ':' in timestamp:
        parts = timestamp.split(':')
        if len(parts) == 2:
            return int(parts[0]) * 60 + float(parts[1])
        elif len(parts) == 3:
            return int(parts[0]) * 3600 + int(parts[1]) * 60 + float(parts[2])
    return float(timestamp)

if __name__ == "__main__":
    print("""
    =========================================
    🎶 Desi Sound Guys - Clean Audio Mashup Creator 🎶
    =========================================
    DJ: DJ Krish
    -----------------------------------------
    Creates smooth transitions between song sections
    without adding any additional beats or effects.
    """)
    song_data_list = []
    while True:
        try:
            num_songs = int(input("\nHow many songs to include in mashup? (2-5 recommended): "))
            if 2 <= num_songs <= 5:
                break
            print("Please enter a number between 2 and 5.")
        except ValueError:
            print("Invalid input. Please enter a number.")
    for i in range(num_songs):
        print(f"\n--- Entering details for Song {i+1} ---")
        while True:
            url = input("YouTube URL: ").strip()
            if url.startswith(('http://', 'https://')):
                break
            print("Please enter a valid URL starting with http:// or https://")
        while True:
            start_input = input("Start timestamp (MM:SS or seconds, leave blank for beginning): ").strip()
            if not start_input:
                start_sec = None
                break
            try:
                start_sec = parse_timestamp(start_input)
                break
            except:
                print("Invalid format. Use MM:SS or seconds (e.g., 1:23 or 83).")
        while True:
            end_input = input("End timestamp (MM:SS or seconds, leave blank for end): ").strip()
            if not end_input:
                end_sec = None
                break
            try:
                end_sec = parse_timestamp(end_input)
                if start_sec is not None and end_sec <= start_sec:
                    print("End time must be after start time.")
                    continue
                break
            except:
                print("Invalid format. Use MM:SS or seconds.")
        song_data_list.append({'url': url, 'start': start_sec, 'end': end_sec})

    # Ask for output file name
    while True:
        output_filename = input("\nEnter desired output file name (without .mp3): ").strip()
        if output_filename:
            if not output_filename.lower().endswith('.mp3'):
                output_filename += '.mp3'
            break
        print("Please enter a valid file name.")

    print("\nCreating your clean mashup...")
    mashup_path = create_mashup(song_data_list, output_filename)
    try:
        from google.colab import files
        files.download(mashup_path)
    except ImportError:
        print(f"\nYou can find your mashup at: {os.path.abspath(mashup_path)}")



    🎶 Desi Sound Guys - Clean Audio Mashup Creator 🎶
    DJ: DJ Krish
    -----------------------------------------
    Creates smooth transitions between song sections
    without adding any additional beats or effects.
    



How many songs to include in mashup? (2-5 recommended):  3



--- Entering details for Song 1 ---


YouTube URL:  https://youtu.be/bSAlE_WgHxY
Start timestamp (MM:SS or seconds, leave blank for beginning):  00:10
End timestamp (MM:SS or seconds, leave blank for end):  00:45



--- Entering details for Song 2 ---


YouTube URL:  https://youtu.be/E_SbwSe15y0
Start timestamp (MM:SS or seconds, leave blank for beginning):  01:38
End timestamp (MM:SS or seconds, leave blank for end):  02:14



--- Entering details for Song 3 ---


YouTube URL:  https://youtu.be/BcSejVIxB0E
Start timestamp (MM:SS or seconds, leave blank for beginning):  01:12
End timestamp (MM:SS or seconds, leave blank for end):  01:44

Enter desired output file name (without .mp3):  karim-mix-v2



Creating your clean mashup...

Processing Song 1
Loaded 'downloads/e210932a_RANGISARI (Video) ｜ JugJugg Jeeyo ｜ Varun D, Kiara A, Anil K, Neetu K ｜ Kanishk & Kavita ｜.mp3' - Duration: 35.00s

Processing Song 2
Loaded 'downloads/cc807be9_Ve Haaniyaan - Official Video ｜ Ravi Dubey & Sargun Mehta ｜ Danny ｜ Avvy Sra ｜ Dreamiyata Music.mp3' - Duration: 36.00s

Processing Song 3
Loaded 'downloads/ad1ce14d_Ishq Hai (Official Music Video) ｜ Mismatched Season 3 ｜ A Netflix Series ｜ Anurag Saikia.mp3' - Duration: 32.00s

Creating transition between segments 1 and 2

Creating transition between segments 2 and 3

🎉 Clean mashup created successfully at: /Users/krishmoodbidri/load-test/mixes/output/karim-mix-v2.mp3

You can find your mashup at: /Users/krishmoodbidri/load-test/mixes/output/karim-mix-v2.mp3
